# Slice timing exercise

See: [slice timing correction](https://textbook.nipraxis.org/slice_timing.html)
for the background to this exercise.

In [ ]:
#: Import common modules
import numpy as np  # the Python array package
np.set_printoptions(precision=4, suppress=True)  # print to 4 DP
import matplotlib.pyplot as plt  # the Python plotting package
#: Set defaults for plotting
plt.rcParams['image.cmap'] = 'gray'

import nibabel as nib

import nipraxis

Load the image `ds114_sub009_t2r1.nii` with nibabel. Get the data:

In [ ]:
# Run this cell.
# Load the image 'ds114_sub009_t2r1.nii' with nibabel
# Get the data array from the image
# Fetch image file.
bold_fname = nipraxis.fetch_file('ds114_sub009_t2r1.nii')
img = nib.load(bold_fname)
data = img.get_fdata()
data.shape

As you remember, the first volume in this dataset is a lot different from the
rest, and this will mess up our interpolation in time.

So, we need to remove the first volume from the data first, using slicing:

In [ ]:
# Run this cell.
# Remove the first volume by slicing
fixed_data = data[..., 1:]
fixed_data.shape

We start with example time-courses from the first and second slice.

Use slicing to get a z slice 0 time series for an example voxel at voxel
coordinates (23, 19, 0).

Do the same for a z slice 1 time series from (23, 19, 1).

Plot these time series against volume number on the same graph:

In [ ]:
#- Slice out time series for voxel (23, 19, 0)
slice_0_ts = ...
#- Slice out time series for voxel (23, 19, 1)
slice_1_ts = ...
#- Plot both these time series against volume number, on the same graph
plt...
plt...

In [ ]:
assert len(slice_0_ts) == 172
assert len(slice_1_ts) == 172
assert not np.all(slice_0_ts == slice_1_ts)

The scanner collected slices for these data in an “ascending interleaved”
order. That is, the scanner first collected z slice 0, then z slice 2, up to z
slice 28. It then went back to collect z slice 1, 3, 5 up to z slice 29.

That means the scanner started collecting slice 0 in each volume, at the
beginning of the volume.

The TR (time to collect one volume) is 2.5 seconds.

In [ ]:
# Run this cell.
# The time between scans
TR = 2.5

Make a 1D time vector that has one element per remaining 3D volume in `data`,
that corresponds to the start time in seconds of each volume. This also gives
the slice 0 start times.

In [ ]:
#- Make time vector containing start times in second of each volume,
#- relative to start of first volume.
slice_0_times = ...
# Show the result
slice_0_times

In [ ]:
assert len(slice_0_times) == 172
assert np.all(slice_0_times[[0, -1]] == [0, 171 * TR])

The scanner starts to collect z slice 1 exactly half way through the volume
(half way through the TR). Make a new vector that is the start time of
acquisition of slice 1.

In [ ]:
#- Make time vector containing start times in seconds of z slice 1,
#- relative to start of first volume.
#- Call this `slice_1_times`
slice_1_times = ...
# Show the result
slice_1_times

In [ ]:
assert len(slice_1_times) == 172
assert np.all(slice_1_times[[0, -1]] == [1.25, 171.5 * TR])

Now plot the first 10 values for the slice 0 times, against the first 10 values
of the slice 0 time series.

Do the same plot for the first 10 values of the slice 1 times, against the
first 10 values of the slice 1 time series.

Use the `:+` line marker for the plots to get the actual position of the
points, and dotted lines between them.

In [ ]:
#- Plot first 10 values of slice 0 times against first 10 of slice 0
#- time series;
#- Plot first 10 values of slice 1 times against first 10 of slice 1
#- time series.
#- Use ':+' marker
plt...
plt...

Import `InterpolatedUnivariateSpline` from `scipy.interpolate`. Make a new
linear (`k=1`) interpolation object for slice 1, with the slice 1 times and
values.

In [ ]:
#- Import `InterpolatedUnivariateSpline` from `scipy.interpolate`
#- Make a new linear (`k=1`) interpolation object for slice 1, with
#- slice 1 times and values.
from scipy... import ...
interp = ...
# Show the result
interp

In [ ]:
assert interp is not ...
# Look for a characteristic method of the interp class.
assert hasattr(interp, 'derivative')

Call the object you got with the slice 0 times, to get the estimated time
series values for slice 1, if slice 1 had been collected at the same time as
slice 0:

In [ ]:
#- Call interpolator with `slice_0_times` to get estimated values
slice_1_ts_est = ...
# Show the result
slice_1_ts_est

In [ ]:
assert len(slice_1_ts_est) == 172
assert not np.all(slice_1_ts_est == slice_1_ts)

Repeat the plot of the first 10 values of the time series. This time, on the
same plot, add the estimated values for slice 1, if they had been collected at
the same time as slice 0. Use a black `x` for the estimated points (marker
`'kx'`):

In [ ]:
#- Plot first 10 values of slice 0 times against first 10 of slice 0
#- time series;
#- Plot first 10 values of slice 1 times against first 10 of slice 1
#- time series;
#- Plot first 10 values of slice 0 times against first 10 of
#- interpolated slice 1 time series.
plt..
plt..
plt..

Use numpy to make a new copy of the data matrix. This will contain the slice
time corrected values for all voxels. Copying the data matrix will give us the
data we want for slice 0, because we want to keep the values for z slice 0
unchanged.  We need to make a copy of the array to make sure we do not
overwrite the original data.

In [ ]:
#- Copy old data to a new array
new_data = ...
new_data.shape

In [ ]:
assert np.all(new_data == fixed_data)
# Check it is actually a copy, not another reference to
# the same data.
assert not np.shares_memory(new_data, fixed_data)

Loop over every x voxel coordinate, and then loop over every y voxel
coordinate.

For each x, y voxel coordinate:

* Extract the time series at this x, y coordinate for slice 1.
* Make a linear interpolator object with the slice 1 times and the extracted
  time series.
* Resample this interpolator at the slice 0 times.
* Put this new resampled time series into the new data at the same position.

In [ ]:
#- loop over all x coordinate values
#- loop over all y coordinate values
#- extract the time series at this x, y coordinate for slice 1;
#- make a linear interpolator object with the slice 1 times and the
#- extracted time series;
#- resample this interpolator at the slice 0 times;
#- put this new resampled time series into the new data at the same
#- position.
...
new_data.shape

In [ ]:
assert not np.all(new_data[:, :, 1] == fixed_data[:, :, 1])

Now we need to do the same thing for all the z slices.

To do this, we want to construct an offset vector (call it `time_offset`) of
length (number of z slices) such that adding the `time_offset[z]` to the
acquisition time of the first slice will give us the time of acquisition of
slice `z`. The next few steps are to get to that `time_offset` vector.

First get the number of slices (planes) in each 3D volume

In [ ]:
# Run this cell
n_slices = fixed_data.shape[-2]
n_slices

Next, make a new vector `acquisition_order` that is length `n_slices`, where
`acquisition_order[i]` is the order of acquisition of slice index `i`. For
example, the first 4 elements of `acqusition_order` should be 0, 15, 1, 16.

In [ ]:
#- Make acquisition_order vector, length 30, with values:
#- 0, 15, 1, 16 ... 14, 29
acquisition_order = ...
...
...
# Show the result
acquisition_order

In [ ]:
assert len(acquisition_order) == n_slices
assert np.all(acquisition_order[:4] == [0, 15, 1, 16])

Divide the acquisition order vector by number of slices, and multiply by the
TR, to get the time offset for each z slice, relative to the start of the
scan:

In [ ]:
#- Divide acquisition_order by number of slices, multiply by TR
time_offsets = ...
# Show the result
time_offsets

In [ ]:
assert len(time_offsets) == n_slices
assert np.all(time_offsets[[0, 1, -1]] == [0, 1.25, 2.5 / 30 * 29])

Now we can do our whole slice time correction, for every slice.

* For each z coordinate (slice index):
    * Make a time vector by adding the slice time offset for this slice, to the
      `slice_0` times. Call this the `slice_z_times` vector;
    * For each x coordinate:
        * For each y coordinate:
            * extract the time series at this x, y, z coordinate;
            * make a linear interpolator object with the `slice_z_times` and
              the extracted time series;
            * resample this interpolator at the slice 0 times;
            * put this new resampled time series into the new data at the same
              position.

In [ ]:
#- For each z coordinate (slice index):
#- # Make `slice_z_times` vector for this slice
#- ## For each x coordinate:
#- ### For each y coordinate:
#- #### extract the time series at this x, y, z coordinate;
#- #### make a linear interpolator object with the `slice_z_times` and
#-      the extracted time series;
#- #### resample this interpolator at the slice 0 times;
#- #### put this new resampled time series into the new data at the
#-      same position
...
...

Congratulations - you have just done slice timing correction on this 4D image.